## 语言模型训练数据

主要介绍大模型训练数据的来源、预处理方法、预训练数据对大模型的影响以及常见开源数据集

### 数据来源

大模型训练所需的数据来源主要分为通用数据和专业数据两大类。`通用数据`包括网页、图书、新闻和对话文本等内容。通用数据具有规模达、多样性和易获取等特点，因此可以支持大语言模型的构建语言建模和泛化能力。`专业数据`包括多语言数据、科学数据、代码以及领域特有资料等数据。通过在预训练阶段引入专业数据可以有效提供大语言模型的任务解决能力。目前主流大模型在训练类型分布上的差距很大:  

<img src="./images/data_fenbu.png" style="zoom:60%;" /> 

### 数据处理

大语言模型的相关研究表明，数据质量对于模型的影响很大，因此在收集到数据时需要进行处理。典型的数据处理过程主要包括质量过滤、冗余去除、隐私消除、词元切分等几个步骤。  

<img src="./images/data_precess.png" style="zoom:60%;" /> 

#### 低质过滤

训练数据低质过滤方法大致可以分为两类: 基于分类器和基于启发式的方法。  

基于分类器的方法目标是训练文本质量判断模型(通常是比较简单的线性模型)，并利用该模型识别并过滤低质量数据。主要是使用精选文本通过分类器进行训练，目标是将与训练数据类似的网页给定较高的分数。  

基于启发式的方法则通过一组精心设计的规则来消除低质量文本:  
- 语言过滤：如果模型仅关注一种或者几种语言，那么就可以大幅度的过滤掉数据中其他语言的文本。
- 指标过滤：利用评测指标也可以过滤低质量文本。
- 统计特征过滤：针对文本内容可以计算包括标点符号分布、符号字比(、句子长度等等在内的统计特征，利用这些特征过滤低质量数据。
- 关键词过滤：根据特定的关键词集，可以识别和删除文本中的噪声或无用元素，例如，HTML标签、超链接以及冒犯性词语等。

#### 冗余去除

训练语料库中的重复数据会降低语言模型的多样性，并可能导致训练过程不稳定，从而影响模型性能，因此需要去除冗余部分。冗余去除就是在不同的粒度上进行去除重复内容，包括句子、段落以及文档等粒度的重复。  


在句子级别，首先将文档转换为一个超长的字符序列，然后构造序列的后缀数组。根据数据，可以很容易的找到相同的子串，这样就可以过滤文档间超过一定长度的相同字符串。

在文档级别上，大部分语言模型都是依靠文档间的表面特征相似度(例如n-gram)进行检测并删除重复文档。  

#### 隐私消除

由于很多预训练数据来源互联网，因此可能包含涉及敏感或个人信息的用户生成内容，这可能会增加隐私泄露的风险，因此需要从语料库中删除或替换包含个人身份信息的内容。最直接的方法是采用基于规则的方法，比如采用基于命名实体识别的方法，检测姓名、地址和电话号码等个人信息内容并进行删除或者替换。  
[实体识别介绍](https://blog.csdn.net/kevinjin2011/article/details/124691670)  

#### 词元切分

在传统的nlp中通常以词为基本处理单元，模型都依赖预先设置的词表。在使用中，如果遇到未登录词，模型无法为其生成对应的表示，就会给予其一个默认的通用表示，通常是`[UNK]`。  

此外，基于固定词表的词表示模型对词表大小的选择比较敏感。当词表大小过小时，未登录词的比例较高，影响模型性能。而当词表大小过大时，大量低频词出现在词表中，而这些词的词向量很难得到充分学习。理想模式下，词表示模型应能覆盖绝大部分的输入词，并避免词表过大所造成的数据稀疏问题。  

为了缓解未登录词的问题，一些工作通过利用亚词级别的信息构造词表示向量。研究者提出了子词词元化方法，词元表示模型会维护一个词元词表，其中存在着单词和子词(单词部分信息)。对于输入序列，词元表示模型将每个词拆分为词表内的词元，随后查询每个词元的表示并将输入重新组成为词元表示序列，当下游模型需要计算一个单词或词组的表示时，可以将对应范围内的词元表示合成为需要的表示。词元分析(Tokenization)目标是将原始文本分割成由词元(Token)序列的过程。词元切分也是数据预处理中至关重要的一步。  

字节对编码(Byte Pair Encoding, BPE)模型是一种常见的子词词元模型，其采用的词表包含最常见的单词以及高频出现的子词。在使用中，常见词通常本身位于BPE词表中，而罕见词通常能被分解为若干个包含在 BPE 词表中的词元，从而大幅度降低未登录词的比例。BPE算法包括两个部分：1.词元词表的确定；2.全词切分为词元以及词元合并为全词的方法。  

<img src="./images/bpe.png" style="zoom:60%;" /> 

根据BPE模型的计算过程可知，其主要是统计二元组的频次，然后根据符号合并表合并高频词的二元组，接着再次统计二元组的频次。。。知道达到预设符号表的大小或者是没有二元组可以被合并。  

WordPiece也是一种常见的词元分析算法，也是Bert的分词器算法。该算法的思想与BPE非常相似，都是通过迭代地合并连续的词元，但在合并的选择标准上略有不同。WordPiece需要首先训练一个语言模型，并用该语言模型对所有可能的词元对进行评分。在每次合并时，选择使得训练数据似然概率增加最多的词元对。当然还有更直观的选择度量方法:  

<img src="./images/wordpiece.png" style="zoom:80%;" /> 

> BPE算法代码

In [9]:
from transformers import AutoTokenizer
from collections import defaultdict

corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.", 
]

# 使用GPT-2的Tokenizer将输入分解为单词
tokenizer = AutoTokenizer.from_pretrained('gpt2')

word_freqs = defaultdict(int)

# 统计词频
for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1
    
# 计算基础词典，使用corpus中的所有字符
alphabet = []

for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

# 增加特殊Token在字典的开头 <|endoftext|>
vocab = ["|endoftext|"] + alphabet.copy()

# 将单词切分为字符
splits = {word: [c for c in word] for word in word_freqs.keys()}

# 计算字典中所有词元对的频率
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

# merge_pair 函数用于合并词元对
def merge_pair(a, b, splits):
    for word in word_freqs.keys():
        split = splits[word]
        if len(split) == 1:
            continue
        
        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

# 迭代训练，不断合并词元对，直到达到预设置的词表大小
vocab_size = 50

merges = defaultdict(str)

while len(vocab) < vocab_size:
    pair_freq = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freq.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

# 训练完成，对给定文本进行词元切分
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2:]
                else:
                    i += 1
            splits[idx] = split
    return sum(splits, [])

tokenize("This is not a token.")

['This', 'Ġis', 'Ġ', 'n', 'o', 't', 'Ġa', 'Ġtoken', '.']

In [11]:
# 使用Huggingface的transformer类
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
res = tokenizer.tokenize("This is not a token.")
print(res)

['this', 'is', 'not', 'a', 'token', '.']


### 数据影响分析

这部分不再赘述，当然是训练数据集越大越好，同时数据集与模型参数也是绑定的，小数据集更适用于小模型少量参数模型，训练的效果也更好。同时上文说的质量问题也会显著影响模型的训练效果。

### 开源数据集合

- Pile: Pile数据集是一个用于大语言模型训练的多样性大规模文本语料库，由22个不同的高质量子集构成，包括现有的和新构建的，许多来自学术或专业来源。这些子集包括Common Crawl、Wikipedia、OpenWebText、ArXiv、PubMed 等。Pile的特点是包含了大量多样化的文本，涵盖了不同领域和主题，从而提高了训练数据集的多样性和丰富性。Pile数据集总计规模大小有825GB英文文本。

- ROOTS: ROOTS数据集合是BigScience项目在训练具有1760亿参数的BLOOM大语言模型所使用的数据集合。该数据集合包含46种自然语言和13种编程语言，总计59种语言，整个数据集的大小约1.6TB。

- RefinedWeb: 其主要由从CommonCrawl数据集过滤的高质量数据组成。CommonCrawl数据集包含自2008年以来爬取的数万亿个网页，由原始网页数据、提取的元数据和文本提取结果组成，总数据量超过1PB 